# Train baseline model for nodule classification

## Setup environment

### [Optional]: Install dependencies

In [ ]:
!pip install "monai[nibabel,skimage,pillow,tqdm]" pytorch_lightning wandb

### [Optional]: Download data

In [ ]:
!mkdir -p ../data/full/processed/images ../data/full/processed/masks ../data/full/cache

In [ ]:
!gsutil cp gs://lung-cancer-detection/lidc-idri/processed/nodules.zip ../data/full/processed

In [ ]:
!gsutil cp gs://lung-cancer-detection/lidc-idri/processed/meta.zip ../data/full/processed

In [ ]:
!gsutil cp gs://lung-cancer-detection/lidc-idri/splits.zip ../data/full/

In [ ]:
!unzip ../data/full/processed/nodules.zip -d ../data/full/processed

In [ ]:
!unzip ../data/full/processed/meta.zip -d ../data/full/processed

In [ ]:
!unzip ../data/full/splits.zip -d ../data/full

### [Optional]: Enable module import

In [ ]:
!ln -s ./../lung_cancer_detection

## Load modules and configuration

### Import modules

In [1]:
from pathlib import Path

import numpy as np
import wandb
import torch
from monai.networks.nets import DenseNet
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger

from lung_cancer_detection.data.nodule import ClassificationDataModule
from lung_cancer_detection.models.classification import NoduleClassificationModule
from lung_cancer_detection.utils import load_config, load_json, preview_dataset

### Load configuration file

In [2]:
cp = Path("../configs/cloud.yaml").absolute()
cp.exists()

True

In [3]:
config = load_config(cp)
config

{'random_seed': 47,
 'wandb': {'offline': False, 'project': 'lung-cancer-detection'},
 'artifacts': {'data': {'name': 'lidc-idri-raw',
   'version': 'v1',
   'type': 'dataset',
   'description': 'Zipped dataset of all chest CT scans, masks and nodule volumes in npy format, including scan and nodule metadata. Updated with new volume size for nodules.'},
  'train': {'name': 'lidc-train',
   'version': 'v1',
   'type': 'dataset',
   'description': 'List of patient IDs included in the training set'},
  'valid': {'name': 'lidc-valid',
   'version': 'v1',
   'type': 'dataset',
   'description': 'List of patient IDs included in the validation set'},
  'class_model': {'name': 'nodule-classification-model',
   'version': 'v0',
   'type': 'model',
   'description': 'Basic DenseNet for classifying lung nodules regarding their malignancy'},
  'seg_model': {'name': 'nodule-segmentation-model',
   'version': 'v1',
   'type': 'model',
   'description': 'Basic UNet for segmenting lung nodules in chest

## Explore datasets

In [4]:
splits = [
    load_json(Path(config["data"]["split_dir"])/"train.json"), 
    load_json(Path(config["data"]["split_dir"])/"valid.json")
]
label_mapping = ([1,2,3,4,5], [0,0,0,1,1])

In [5]:
dm = ClassificationDataModule(
    data_dir=Path(config["data"]["data_dir"]),
    cache_dir=(Path()/"../data/cache/").absolute(),
    splits=splits,
    min_anns=config["data"]["min_anns"],
    exclude_labels=[],
    label_mapping=label_mapping,
    batch_size=config["data"]["batch_size"]
)

In [6]:
dm.setup()

In [7]:
print(f"Number of training examples: {len(dm.train_ds)}")
print(f"Number of validation examples: {len(dm.val_ds)}")

Number of training examples: 2155
Number of validation examples: 470


In [8]:
train_labels = np.array([image["label"].numpy() for image in dm.train_ds])
valid_labels = np.array([image["label"].numpy() for image in dm.val_ds])

In [9]:
print(f"Percentage of malignant training nodules: {np.mean(train_labels):.4f}")
print(f"Percentage of malignant training nodules: {np.mean(valid_labels):.4f}")

Percentage of malignant training nodules: 0.2158
Percentage of malignant training nodules: 0.2234


## Validation check: overfit one batch

In [10]:
dm = ClassificationDataModule(
    data_dir=Path(config["data"]["data_dir"]),
    cache_dir=(Path()/"../data/cache/").absolute(),
    splits=splits,
    min_anns=config["data"]["min_anns"],
    exclude_labels=[],
    label_mapping=label_mapping,
    batch_size=config["data"]["batch_size"]
)

In [11]:
net = DenseNet(
    spatial_dims=config["class_model"]["spatial_dims"],
    in_channels=config["class_model"]["in_channels"],
    out_channels=config["class_model"]["out_channels"],
)

In [12]:
model = NoduleClassificationModule(net, num_classes=config["class_model"]["num_classes"])

In [13]:
trainer = Trainer(gpus=1, overfit_batches=1, max_epochs=20)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [14]:
trainer.fit(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | DenseNet         | 11.2 M
1 | loss      | CrossEntropyLoss | 0     
2 | train_acc | Accuracy         | 0     
3 | val_acc   | Accuracy         | 0     
-----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.979    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:342: UserWarning: You requested to overfit but enabled val/test dataloader shuffling. We are turning it off for you.
  'You requested to overfit but enabled val/test dataloader shuffling.'
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:240: UserWarning: You requested to overfit but enabled training dataloader shuffling. We are turning it off for you.
  'You requested to overfit but enabled training dataloader shuffling.'


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

## Run experiment

### Configure experiment

In [15]:
dm = ClassificationDataModule(
    data_dir=Path(config["data"]["data_dir"]),
    cache_dir=(Path()/"../data/cache/").absolute(),
    splits=splits,
    min_anns=config["data"]["min_anns"],
    exclude_labels=[],
    label_mapping=label_mapping,
    batch_size=config["data"]["batch_size"]
)

In [16]:
net = DenseNet(
    spatial_dims=config["class_model"]["spatial_dims"],
    in_channels=config["class_model"]["in_channels"],
    out_channels=config["class_model"]["out_channels"],
)
model = NoduleClassificationModule(net, num_classes=config["class_model"]["num_classes"], lr=1e-5)

In [17]:
wandb.login()

wandb: Currently logged in as: felixpeters (use `wandb login --relogin` to force relogin)


True

In [18]:
logger = WandbLogger(project=config["wandb"]["project"], job_type="training")

In [19]:
es = EarlyStopping(monitor="val_loss", verbose=True)
mc = ModelCheckpoint(monitor="val_loss", filename="{epoch}-{step}-{val_loss:.4f}-{val_acc:.4f}", verbose=True, save_top_k=1)
callbacks = [es, mc]

In [20]:
trainer = Trainer(
    logger=logger,
    callbacks=callbacks,
    **config["experiment"]
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [21]:
trainer.logger.experiment.use_artifact(config["artifacts"]["train"]["name"] + ":" + config["artifacts"]["train"]["version"])
trainer.logger.experiment.use_artifact(config["artifacts"]["valid"]["name"] + ":" + config["artifacts"]["valid"]["version"])

<Artifact QXJ0aWZhY3Q6NDIzMjE4OA==>

### Train model

In [22]:
trainer.fit(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | DenseNet         | 11.2 M
1 | loss      | CrossEntropyLoss | 0     
2 | train_acc | Accuracy         | 0     
3 | val_acc   | Accuracy         | 0     
-----------------------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.979    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:349: UserWarning: Your val_dataloader has `shuffle=True`, it is best practice to turn this off for val/test/predict dataloaders.
  f'Your {mode}_dataloader has `shuffle=True`, it is best practice to turn'


Training: 0it [00:00, ?it/s]

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/callback_hook.py:101: LightningDeprecationWarning: The signature of `Callback.on_train_epoch_end` has changed in v1.3. `outputs` parameter has been removed. Support for the old signature will be removed in v1.5
  "The signature of `Callback.on_train_epoch_end` has changed in v1.3."


...

Epoch 17, global step 305: val_loss was not in top 1


Validating: 0it [00:00, ?it/s]

Epoch 18, global step 322: val_loss was not in top 1


Validating: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 3 records. Best score: 0.512. Signaling Trainer to stop.
Epoch 19, global step 339: val_loss was not in top 1


### Finish experiment

In [23]:
model_artifact = wandb.Artifact(
    config["artifacts"]["class_model"]["name"],
    type=config["artifacts"]["class_model"]["type"],
    description=config["artifacts"]["class_model"]["description"],
)
model_artifact.add_file(mc.best_model_path)
trainer.logger.experiment.log_artifact(model_artifact)

In [24]:
wandb.finish()

val_loss,0.51524
val_acc,0.78936
epoch,19
trainer/global_step,339
_runtime,225
_timestamp,1625830962
_step,25
train_loss,0.11314
train_acc,1.0


val_loss,██▅▄▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁
val_acc,▁▁▆▇▇▇██████████████
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
trainer/global_step,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
_runtime,▁▁▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
_timestamp,▁▁▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
_step,▁▁▂▂▂▂▃▃▃▄▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
train_loss,█▆▅▃▂▁
train_acc,▁▆▆▇██
